In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import seaborn as sns
from datetime import datetime, date, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



In [2]:
df= pd.read_excel(r'C:\Users\basia\OneDrive\Pulpit\sdu shit\thesis\READY DATASETS\median.xlsx')
df.head()


,datetime,Origin,Day,Month,Week,Weekday,Hour,Minute,Price,Output
0,2022-01-01 15:00:00,1,1,1,1,1,15,0,349.0,55
1,2022-01-01 16:15:00,0,1,1,1,1,16,15,349.0,25
2,2022-01-02 07:00:00,1,2,1,1,2,7,0,399.0,44
3,2022-01-02 08:15:00,0,2,1,1,2,8,15,399.0,13
4,2022-01-02 15:15:00,1,2,1,52,6,15,15,399.0,25


In [3]:
df.shape

(4099, 10)

In [4]:
# Convert the 'datetime' column to datetime type
df['datetime'] = pd.to_datetime(df['datetime'])

# Set the 'datetime' column as the index
df.set_index('datetime', inplace=True)

# Extract exogenous features and target variable
exogenous_features = df[['Origin', 'Day', 'Month', 'Week', 'Weekday', 'Hour', 'Minute', 'Price']]
target_variable = df['Output']


In [6]:
# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the exogenous features
exogenous_features_scaled = scaler.fit_transform(exogenous_features)

# Normalize the target variable
target_variable_scaled = scaler.fit_transform(target_variable.values.reshape(-1, 1))


In [14]:
# Set the train-test split ratio
split_ratio = 0.8

# Determine the split index
split_index = int(len(df) * split_ratio)

# Split the exogenous features and target variable
exogenous_train = exogenous_features_scaled[:split_index]
exogenous_test = exogenous_features_scaled[split_index:]
target_train = target_variable_scaled[:split_index]
target_test = target_variable_scaled[split_index:]


In [15]:
exogenous_train.shape

(3279, 8)

In [16]:
# Define the number of time steps and features
time_steps = 10
num_features = exogenous_features.shape[1]

# Function to create input sequences
def create_sequences(X, y, time_steps):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i + time_steps])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

# Create input sequences for training and test sets
X_train, y_train = create_sequences(exogenous_train, target_train, time_steps)
X_test, y_test = create_sequences(exogenous_test, target_test, time_steps)


In [17]:
# Define the model architecture
model = Sequential()
model.add(LSTM(units=64, input_shape=(time_steps, num_features), return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
103/103 [==============================] - 6s 12ms/step - loss: 0.0356
Epoch 2/100
103/103 [==============================] - 1s 12ms/step - loss: 0.0289
Epoch 3/100
103/103 [==============================] - 1s 10ms/step - loss: 0.0283
Epoch 4/100
103/103 [==============================] - 1s 10ms/step - loss: 0.0283
Epoch 5/100
103/103 [==============================] - 1s 12ms/step - loss: 0.0274
Epoch 6/100
103/103 [==============================] - 1s 11ms/step - loss: 0.0268
Epoch 7/100
103/103 [==============================] - 1s 12ms/step - loss: 0.0265
Epoch 8/100
103/103 [==============================] - 1s 12ms/step - loss: 0.0263
Epoch 9/100
103/103 [==============================] - 1s 11ms/step - loss: 0.0258
Epoch 10/100
103/103 [==============================] - 1s 11ms/step - loss: 0.0258
Epoch 11/100
103/103 [==============================] - 1s 12ms/step - loss: 0.0256
Epoch 12/100
103/103 [==============================] - 1s 11ms/step - loss: 0.0254
E

In [18]:
# Generate predictions on the test set
predictions = model.predict(X_test)

# Rescale the predictions and target variable
predictions_rescaled = scaler.inverse_transform(predictions)
target_test_rescaled = scaler.inverse_transform(target_variable_scaled[split_index + time_steps:].reshape(-1, 1))

# Calculate the root mean squared error (RMSE)
rmse = np.sqrt(np.mean((predictions_rescaled - target_test_rescaled) ** 2))
print('RMSE:', rmse)


RMSE: 10.440987826250117


In [19]:
from sklearn.metrics import mean_squared_error
mean_squared_error(predictions_rescaled, target_test_rescaled) #

109.01422678790314

In [13]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(predictions_rescaled, target_test_rescaled) #

9.873328012075179

In [158]:
# Rescale the training output, predicted predictions, and test output
train_output_rescaled = scaler.inverse_transform(target_train)
predicted_predictions_rescaled = scaler.inverse_transform(predictions.reshape(-1, 1))
test_output_rescaled = scaler.inverse_transform(target_test[time_steps:].reshape(-1, 1))

# Concatenate the arrays
result = np.concatenate(( predicted_predictions_rescaled, test_output_rescaled), axis=1)

# Create a DataFrame with the concatenated results
df_result = pd.DataFrame(result, columns=[ 'Predicted Prediction', 'Test Output'])
print(df_result)


     Predicted Prediction  Test Output
0               13.371153         16.0
1               17.925447         20.0
2               19.706711         24.0
3               13.115160         10.0
4               24.100447         19.0
..                    ...          ...
800             16.143347         10.0
801             11.728500         51.0
802             22.544617         43.0
803             15.075020         27.0
804             19.486526         17.0

[805 rows x 2 columns]


In [163]:
train_output_rescaled.shape
df.index[3279:].shape

predicted_predictions_rescaled.shape
#test_output_rescaled.shape

(805, 1)

In [167]:
train_output_rescaled=pd.DataFrame(train_output_rescaled)

In [169]:
n_steps=10
line = train_output_rescaled.rolling(n_steps).mean()